In [ ]:
with open("../raw_results/omcs/filtered/subjects_filtered.txt", 'r') as file1:
    subjs = [line.split(':')[0] for line in file1.readlines()]

subjs

In [ ]:
with open("../raw_results/omcs/filtered/gerund_phrases_filtered.txt", 'r') as file2:
    vps = [line.strip() for line in file2.readlines()]

vps

In [ ]:
with open("../raw_results/omcs/filtered/locations_filtered.txt", 'r') as file3:
    locs = [line.split(':')[0] for line in file3.readlines()]

locs

In [68]:
from lemminflect import getLemma, getInflection

def check_if_singular(subject_noun):
    words = subject_noun.split()
    key_article = words[0]
    key_noun = words[-1]
    if key_article == 'a': # high priority
        return True
    
    singular_set = set()
    lemma_tup = getLemma(key_noun, upos='NOUN')
    for lemma in lemma_tup:
        lemma_singular_tup = getInflection(lemma, 'NN', False)
        for lemma_singular in lemma_singular_tup:
            singular_set.add(lemma_singular)

    return key_noun in singular_set

In [90]:
check_if_singular("parents")

False

In [ ]:
import json

with open("tmp_compositional_sents.json", 'w') as out:
    sent_list = []
    i = 0
    for sub in subjs:
        be_verb = 'is' if check_if_singular(sub) else 'are'
        for loc in locs:
            for vp in vps:
                local = {}
                # who + doing what + at where
                local["id"] = i
                local["subj"] = sub
                local['be'] = be_verb
                local["vp"] = vp
                local["loc"] = loc
                local["text"] = sub + ' ' + be_verb + ' ' + vp + ' ' + loc # e.g. [the policemen] [are] [drinking from a bottle] [in the refrigerator] 
                # print(local["text"])
                sent_list.append(local)
                i += 1
                if (i % 1000 == 0):
                    print(f"Processed {i} sentences")
    print(f"Total: Processed {i} sentences")
    j_f = {'data': sent_list}
    json.dump(j_f, out)